Speed Detection and Car tracking

In [2]:
import sys
import os
sys.path.append(os.path.abspath("../src"))

try :
    from tracker import *
except ImportError :
    print('Import not done')

In [3]:
import ultralytics
import cv2
import pandas as pd
from ultralytics import YOLO


In [4]:
import math


class Tracker:
    def __init__(self):
        # Store the center positions of the objects
        self.center_points = {}
        # Keep the count of the IDs
        # each time a new object id detected, the count will increase by one
        self.id_count = 0


    def update(self, objects_rect):
        # Objects boxes and ids
        objects_bbs_ids = []

        # Get center point of new object
        for rect in objects_rect:
            x, y, w, h = rect
            cx = (x + x + w) // 2
            cy = (y + y + h) // 2

            # Find out if that object was detected already
            same_object_detected = False
            for id, pt in self.center_points.items():
                dist = math.hypot(cx - pt[0], cy - pt[1])

                if dist < 35:
                    self.center_points[id] = (cx, cy)
#                    print(self.center_points)
                    objects_bbs_ids.append([x, y, w, h, id])
                    same_object_detected = True
                    break

            # New object is detected we assign the ID to that object
            if same_object_detected is False:
                self.center_points[self.id_count] = (cx, cy)
                objects_bbs_ids.append([x, y, w, h, self.id_count])
                self.id_count += 1

        # Clean the dictionary by center points to remove IDS not used anymore
        new_center_points = {}
        for obj_bb_id in objects_bbs_ids:
            _, _, _, _, object_id = obj_bb_id
            center = self.center_points[object_id]
            new_center_points[object_id] = center

        # Update dictionary with IDs not used removed
        self.center_points = new_center_points.copy()
        return objects_bbs_ids

In [5]:
model = YOLO('yolov8s.pt')

In [6]:
class_list = ['person', 'bicycle', 'car','motorcycle', 'airplane', 'bus',
              'train', 'truck', 'traffic light']

In [7]:
tracker = Tracker()
count = 0

In [8]:
down = {}

In [13]:
cap = cv2.VideoCapture('C:\\Users\\Aman\\Desktop\\thesisproject\\video\\highway (1).mp4')

In [14]:
while True:
    ret, frame = cap.read()
    if not ret:
        break
    count += 1
    frame = cv2.resize(frame,(1020,500))

    result = model.predict(frame)
    # print(result)
    a = result[0].boxes.data
    a = a.detach().cpu().numpy() # added this line
    px = pd.DataFrame(a).astype("float")
    # print(px)

    list = []

    for index, row in px.iterrows():
      print(row)
      x1 = int(row[0])
      x2 = int(row[2])
      y1 = int(row[1])
      y2 = int(row[3])
      d = int(row[5])
      c = class_list[d]
      if 'car' in c:
        list.append([x1,y1,x2,y2])
        # print(c)
    bbox_id = tracker.update(list)
    # print (bbox_id)
    for bbox in bbox_id:
      x3,y3,x4,y4,id = bbox
      cx = int(x3+x4)//2
      cy = int(y3+y4)//2

      red_line_y = 198
      blue_line_y = 268
      offset = 7
      ''' condition for red line'''
      if red_line_y < (cy + offset) and red_line_y > (cy - offset):
         ''' this if condition is putting the id and the circle on the object'''

         down[id] = cy
         if id in down:
            cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
            cv2.putText(frame,str(id),(cx,cy),cv2.FONT_HERSHEY_COMPLEX,0.8,(o,255,255), 2)
            # cv2.rectangle(frame,(x3,y3),(x4,y4),(0,0,255),2)

            
    text_color = (255,255,255  )
    red_color = (0,0,255)
    blue_color = (255,0,0)
    green_color = (0,255,0)

    cv2.line(frame, (172,198), (774,198), red_color,3)
    cv2.putText(frame,('red line'),(172,198),cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color,1, cv2.LINE_AA)
    cv2.line(frame, (8,268), (927,268), blue_color, 3)
    cv2.putText(frame,('blue line'),(8,268), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)


    cv2.imshow('frames',frame)
    
    if cv2.waitKey(0)& 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


0: 320x640 8 cars, 2 trucks, 369.9ms
Speed: 8.2ms preprocess, 369.9ms inference, 3.3ms postprocess per image at shape (1, 3, 320, 640)
0      0.146739
1    271.796753
2     66.228218
3    329.035400
4      0.855663
5      2.000000
Name: 0, dtype: float64
0    211.525070
1    125.842216
2    295.862213
3    201.701172
4      0.854887
5      7.000000
Name: 1, dtype: float64
0    587.201233
1    225.676514
2    643.098022
3    265.474823
4      0.813440
5      2.000000
Name: 2, dtype: float64
0    722.433777
1    203.575775
2    771.671875
3    237.283051
4      0.789404
5      2.000000
Name: 3, dtype: float64
0    616.325867
1    175.107361
2    655.397278
3    200.931534
4      0.782868
5      2.000000
Name: 4, dtype: float64
0    373.264435
1    134.722702
2    397.598206
3    151.949554
4      0.619406
5      2.000000
Name: 5, dtype: float64
0    448.418182
1    105.682892
2    468.934601
3    118.410034
4      0.301781
5      2.000000
Name: 6, dtype: float64
0    520.832764
1    119

NameError: name 'o' is not defined

: 